## Ingestion de la carpeta "movie_company"


#### Paso 0 - Actualización de variables y funciones y validaciones

In [0]:
dbutils.widgets.text("p_environment", "")
v_environment = dbutils.widgets.get("p_environment")

In [0]:
dbutils.widgets.text("p_file_date", "2024-12-16")
v_file_date = dbutils.widgets.get("p_file_date")

In [0]:
%run "../includes/configuration"

In [0]:
%run "../includes/common_functions"

In [0]:
#Chama a função para validar se o arquivo a ser processado existe
valida_arquivo(bronze_folder_path, v_file_date, "movie_company/movie_company_1.csv")

#### Paso 1 - Leer el archivo CSV usando "DataFrameReader" de Spark

In [0]:
#%fs
#ls /mnt/moviehistoryadilmor/bronze/movie_company/

In [0]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

In [0]:
#Definindo o schema na variavel name_schema
movies_companies_schema = StructType(fields=[
    StructField("movie_id", IntegerType(), True),
    StructField("company_id", IntegerType(), True)
])

In [0]:
#Forma 2 - Especificando el nombre del arquivos com "*"
movies_companies_df = spark.read \
               .schema(movies_companies_schema) \
               .csv(f"{bronze_folder_path}/{v_file_date}/movie_company/movie_company_*.csv")

In [0]:
#display(movies_companies_df)
#movies_companies_df.count()

#### Paso 2 - Renombrar el nombre de las columnas y añadir "ingestion_date" y "enviroment"

In [0]:
from pyspark.sql.functions import current_timestamp, lit

In [0]:
movies_companies_final_df = add_ingestion_date(movies_companies_df) \
                    .withColumn("enviroment", lit(v_environment)) \
                    .withColumn("file_date", lit(v_file_date))

In [0]:
#display(movies_companies_final_df)

#### Paso 4 - Escribir datos en el datalake en formato parquet

In [0]:
#Elimina partição se ela já existir
#drop_partition_if_exists(movies_companies_final_df, "movie_silver", "movies_companies", "file_date")

In [0]:
#movies_companies_final_df.write.mode("overwrite").parquet(f"{silver_folder_path}/movies_companies/")
#movies_companies_final_df.write.mode("append") \
#                        .partitionBy("file_date") \
#                        .format("parquet") \
#                        .saveAsTable("movie_silver.movies_companies")

#Caso a tabela não existir, cria a tabela com os dados, senão efetua um merge dos dados na tabela
merge_delta_lake(movies_companies_final_df, "movie_silver", "movies_companies", silver_folder_path, "tgt.movie_id = src.movie_id AND tgt.company_id = src.company_id", "file_date")

In [0]:
#%fs
#ls /mnt/moviehistoryadilmor/silver/movies_companies/

In [0]:
#display(spark.read.parquet(f"{silver_folder_path}/movies_companies/"))
#(spark.read.parquet("/mnt/moviehistoryadilmor/silver/movies_companies/")).count()
#spark.sql("select * from movie_silver.movies_companies").display()
display(spark.sql("SELECT file_date, \
                          COUNT(1) \
                   FROM movie_silver.movies_companies \
                   GROUP BY file_date"
                   )
        )

In [0]:
dbutils.notebook.exit("Success")